# Setting up Twitter Streaming API

I had to register a Twitter Developers account, which I did as a student, it gave me access right away. After that I needed to set up the stream.

In [1]:
import socket
import sys
import requests
from requests_oauthlib import OAuth1
import json

In [2]:
API_KEY = 'mChKtMIlxucUhULo2VsiXXuz1'
API_SECRET = '8wSiOpCdfY7PAq8JDVUHWjB1z6ltxIJ4rGynvBxVEtfD7DA23L'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAHf4LQEAAAAAfTYRa4qkKXl1yHSXzSl%2BL0hfAMU%3DjbbxiTPUuSONxcM6b0I2EOTrhNAVlbi4nTwArftzbxCUzyFe4P'

In [3]:
request_token_url = 'https://api.twitter.com/oauth/request_token'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY, client_secret=API_SECRET)
r = requests.post(url=request_token_url, auth=oauth)
from urllib.parse import parse_qs
credentials = parse_qs(r.content)
resource_owner_key = credentials[b'oauth_token'][0].decode("utf-8") 
resource_owner_secret = credentials[b'oauth_token_secret'][0].decode("utf-8") 

In [4]:
base_authorization_url = 'https://api.twitter.com/oauth/authorize'

# Using OAuth1 auth helper
authorize_url = base_authorization_url + '?oauth_token='
authorize_url = authorize_url + resource_owner_key
print('Please go here and authorize,', authorize_url)

Please go here and authorize, https://api.twitter.com/oauth/authorize?oauth_token=2fxiEAAAAAABLfh3AAABd0qft64


In [5]:
verifier = '3244210'

In [6]:
access_token_url = 'https://api.twitter.com/oauth/access_token'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY,
                   client_secret=API_SECRET,
                   resource_owner_key=resource_owner_key,
                   resource_owner_secret=resource_owner_secret,
                   verifier=verifier)
r = requests.post(url=access_token_url, auth=oauth)
credentials = parse_qs(r.content)
resource_owner_key = credentials.get(b'oauth_token')[0].decode("utf-8")
resource_owner_secret = credentials.get(b'oauth_token_secret')[0].decode("utf-8")

In [7]:
protected_url = 'https://api.twitter.com/1/account/settings.json'

# Using OAuth1 auth helper
oauth = OAuth1(API_KEY,
                   client_secret=API_SECRET,
                   resource_owner_key=resource_owner_key,
                   resource_owner_secret=resource_owner_secret)
r = requests.get(url=protected_url, auth=oauth)

After setting up the API, I created a function, where I could check if it works properly:

In [8]:
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=oauth, stream=True)
    print(query_url, response)
    return response

In [9]:
resp = get_tweets()

i = 0
for line in resp.iter_lines():
    full_tweet = json.loads(line)
    tweet_text = full_tweet['text']
    print("Tweet Text: " + repr(tweet_text))
    print ("------------------------------------------")
    i += 1
    if i == 5:
        break

https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>
Tweet Text: 'Jesus said, "I am giving money no matter what."'
------------------------------------------
Tweet Text: 'We have people who are slow growth, tree advocates, historic preservation people all scheduled to attend our… https://t.co/Ac45rATWxD'
------------------------------------------
Tweet Text: 'Just checking in....⠀\n⠀\nWhat’s your favorite Thai food?  ➖ We are checking out new spots in #downtownbakersfield ⠀… https://t.co/3dj2gtjz7o'
------------------------------------------
Tweet Text: 'Manipulation in broad view.'
------------------------------------------
Tweet Text: 'Just posted a photo @ Lagos, Nigeria https://t.co/R3l3E6nljg'
------------------------------------------


It is working as intended. Now I have to channel the data into Spark.

In [10]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text'] + '\n'
            print("Tweet Text: " + repr(tweet_text))
            print ("------------------------------------------")
            tcp_connection.send((tweet_text).encode('utf-8'))
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

In [11]:
TCP_IP = "localhost"
TCP_PORT = 37346
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

Waiting for TCP connection...
Connected... Starting getting tweets.
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-130,-20,100,50&track=# <Response [200]>
Tweet Text: 'Long time no see!  Let help them cum!\n'
------------------------------------------
Tweet Text: '@bruce_stauffer @MrTAchilles Happy birthday and best wishes!\n'
------------------------------------------
Tweet Text: '🗣 never get revenge because karma always spin the block and work her magic \U0001fa84\n'
------------------------------------------
Tweet Text: '@nerdyncurvy Yes especially if its chocolate. Cake for breakfast is a thing. Spread th is message far and wide. Thi… https://t.co/McjMMpOczu\n'
------------------------------------------
Tweet Text: 'I have a sticker of the UofL speed art museum on my laptop and my sister asked me if it was the Cullen house from twilight\n'
------------------------------------------
Tweet Text: 'My little nephew spotted me in a store and run to say hi. We

Tweet Text: '@mariocamp02 Would’ve had to have money 6 months ago to do so - have prioritized getting out of debt.\n'
------------------------------------------
Tweet Text: 'I call this the "ride em like a dirt bike position".\n'
------------------------------------------
Tweet Text: '@ovppodcast Did they get announced as the mega powers lol? Do you have the intro or finish?\n'
------------------------------------------
Tweet Text: "VAR is only bad when it's not on your team's side https://t.co/y9TKteuLK7\n"
------------------------------------------
Tweet Text: 'I believe with all of my heart that if @BetoORourke runs for TX Governor next year, he would make that race super c… https://t.co/tW2dgcOZgx\n'
------------------------------------------
Tweet Text: 'Your Talent is in choice of Life ...IRIS \nhttps://t.co/0Y3kUM4WAT https://t.co/sibhDdZ3LH\n'
------------------------------------------
Tweet Text: 'Congrats to her however I think I’ld have to take the test like 6 times, add the

Tweet Text: '@AuthorKimberley No organic meals there I bet 😂\n'
------------------------------------------
Tweet Text: 'People who are laughing right now or calling it "it was all meme" you really gonna sit there and deny this didn\'t m… https://t.co/9Fjv2CADmv\n'
------------------------------------------
Tweet Text: '@skgilroy @EmilyGorcenski Oliver Stone definitely framed their JFK experience lol https://t.co/t7ZBhEFl6a\n'
------------------------------------------
Tweet Text: 'Back in stock Mothers Foaming ammonia free window tint glass cleaner #sale #windowtint @ DJS Motorsports https://t.co/rMIUCIOXPr\n'
------------------------------------------
Tweet Text: '@7Scrolls_ LETS FUCKIN GO. https://t.co/SmiAFNbuS0\n'
------------------------------------------
Tweet Text: 'check @lil_meek_lil’s fleets\n'
------------------------------------------
Tweet Text: '@davonyaofficial I would like to see!\n'
------------------------------------------
Tweet Text: '@LMainWeather @cleanrag @justin

Tweet Text: 'murder she wrote 😍\n'
------------------------------------------
Tweet Text: 'My link today is so shitty\n'
------------------------------------------
Tweet Text: 'This job is now open at Cintas Corporation in Indianapolis, IN. Follow us and turn on mobile alerts to hear about j… https://t.co/N5EoX6H4ew\n'
------------------------------------------
Tweet Text: 'Straight pipe 💆🏾\u200d♂️💆🏾\u200d♂️\n'
------------------------------------------
Tweet Text: 'Thank you coach! Yes sir!!\n'
------------------------------------------
Tweet Text: 'I can commit to myself and I will ♒️\n'
------------------------------------------
Tweet Text: '@martinlieberman Hah, hadn’t thought of that angle! https://t.co/RVtPC6KTun\n'
------------------------------------------
Tweet Text: 'Play with mi pussy don’t play with mi heart ! 😉\n'
------------------------------------------
Tweet Text: 'and this is why I need to keep microdosing 🙃\n'
------------------------------------------
Tweet Text: 'F

Tweet Text: '@LunaAmofficial1 Yes and you have my attention\n'
------------------------------------------
Tweet Text: 'With @khalenizi33 🤍\n'
------------------------------------------
Tweet Text: '@scsears_wde Freaking ridiculous.. might as well tie a bag around our head at this point\n'
------------------------------------------
Tweet Text: 'Here’s an album for you… Told Ya Noteboc\nhttps://t.co/n4qV88u4f4\n'
------------------------------------------
Tweet Text: '@AmericanHisto20 @ericbolling 😂 wow. You have strong feelings about cheese\n'
------------------------------------------
Tweet Text: '@rodislavable @linuxtoday Do you have autocompletion? 😁\n'
------------------------------------------
Tweet Text: 'Omo after watching I gats breathe abeg!!! What!!!!\n #SilhoutteChallenge\n'
------------------------------------------
Tweet Text: '“Dont worry about that shirt tug in the box”. \n\nCant wait for Kane to win a penalty because his shirt is tugged. https://t.co/8zxgFuyyBx\n'
------

...

Tweet Text: 'Never thought we’d have to “build the wall” in DC to keep clowns from killing our Congress people, but here we are.… https://t.co/cmLtQY1X9X\n'
------------------------------------------
Tweet Text: 'Even though I don’t support the irrational move (far away from fundamentals) in stocks such as #tesla #GME and othe… https://t.co/g8PoqMvpjf\n'
------------------------------------------
Tweet Text: '@WebullGlobal will you be renaming to Wecitadel?\n'
------------------------------------------
Tweet Text: '@ReneKerstin Thank u so much 🥺🥺🥺🥺\n'
------------------------------------------
Tweet Text: '@RiverRTX69420 my rule is i can make two special requests on an order at a restaurant. anything more is annoying\n'
------------------------------------------
Tweet Text: '@SpeakerPelosi I would add, that GA, by its vote elivatining our party/dems to control of the Senate, did so to bre… https://t.co/acidk2mfF0\n'
------------------------------------------
Tweet Text: 'They get this 

Tweet Text: 'After seeing some of these silhouette challenge videos... Got me thinking which is better that or the Buss it chall… https://t.co/Qn4wcs5TsL\n'
------------------------------------------
Tweet Text: 'Cleared: Incident on #TaconicStateParkway NB at Exit 25 - Peekskill Hollow Rd. - CR 21\n'
------------------------------------------
Tweet Text: 'Cleared: Construction on #StatenIslandRailway Both directions from Old Town to Jefferson Av\n'
------------------------------------------
Tweet Text: 'Such a good explanation for #noviceinvestors @reddit\n'
------------------------------------------
Tweet Text: '@GayLatina4Trump #DavidDorn \nSay his name, @AOC\n'
------------------------------------------
Tweet Text: 'Trump shits on himself regularly!!!\n'
------------------------------------------
Tweet Text: 'Me in this meeting right now https://t.co/0oAUUUbaqY\n'
------------------------------------------
Tweet Text: 'Cleared: Construction on #US19 NB at US 22\n'
-----------------

Tweet Text: '@cannoliini i don’t mind a shiny finish, but you seemed like a matte kinda girl so i said to go with matte.🤣\n'
------------------------------------------
Tweet Text: 'Her voice is trembling 🥺🥺 #nde\n'
------------------------------------------
Tweet Text: "Looking to score a job at UPS? You're in luck. Click the link in our bio for the job description and information on… https://t.co/0JkxoVSAAu\n"
------------------------------------------
Tweet Text: "Age doesn't guarantee sense, too many senseless adults everywhere.\n@BabangidaRuma\n"
------------------------------------------
Tweet Text: '@therealjonnz Ween ask\n'
------------------------------------------
Tweet Text: 'last 4 weeks and last 6 months… genuinely i am so predictable it hurts https://t.co/FjRvclEAr4\n'
------------------------------------------
Tweet Text: '🥱 next\n'
------------------------------------------
Tweet Text: '@Bet9jaOfficial Group c\n'
------------------------------------------
Tweet Text: 'Oh

Tweet Text: 'Paul Anka\n'
------------------------------------------
Tweet Text: 'Anyone seen King Canabis?\n'
------------------------------------------
Tweet Text: 'What a time to be alive.....\n\nAlso... #BellLetsTalk\n'
------------------------------------------
Tweet Text: '@_dontfuckabout So 1 in 6 right?... So that means the remaining 5 out of the 6 men will face the consequences for t… https://t.co/QwXkjqmUvJ\n'
------------------------------------------
Tweet Text: 'Some of y’all pussy taste like hot dog water !!\n'
------------------------------------------
Tweet Text: 'Remember poor people you are only supposed to be focused on guns, abortion, and whatever the LGBT are doing today.… https://t.co/n3mE09emzS\n'
------------------------------------------
Tweet Text: '@Brush THATS MY DUO\n'
------------------------------------------
Tweet Text: 'Free cyber #STEM programs sign up and get your kits delivered ! Each will contain all of the items you need for you… https://t.co/rsKc3

KeyboardInterrupt: 

In [12]:
s.close()